In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import importlib
sys.path.append('../src/')
sys.path.append('../data/')
import model as mod
importlib.reload(mod)
#from data_utils import generate_synthethic, transform_and_normalize, CCCscore
import load_rosmap as lr
importlib.reload(lr)
import data_utils as du
importlib.reload(du)
from sklearn.model_selection import train_test_split
import torch

In [2]:
## lets use the human brain cortex dataset from the original manuscript
scRNA, bulkRNA = lr.load_and_filter_hbc_rosmap(genes_cutoff=3000)

## define number of samples
nsamples = 5000
train_size, test_size = 0.8, 0.2

In [7]:
#split into train and test
scRNA_train, scRNA_test = train_test_split(scRNA.copy(), stratify=scRNA.index, test_size = 0.2, random_state= 42)

# create pseudobulk for train
xtrain, ytrain = du.generate_synthethic(scRNA_train, nsamples = train_size)
xtest, ytest = du.generate_synthethic(scRNA_test, nsamples = test_size)

## transform and normalize
xtrain, xtest, xbulk = du.transform_and_normalize(xtrain, xtest, bulkRNA.values)

## convert to torch
xtrain, ytrain, xtest, ytest, xbulk = du.convert_to_float_tensors(xtrain, ytrain, xtest, ytest, xbulk)

simulating bulk: 100%|██████████| 511/511 [00:00<00:00, 1210.63it/s]


In [8]:
## define epochs and init sweetwater object
epochs = round(30000/(xtrain.shape[0]/256))
sw = mod.SweetWater(data = (xtrain, ytrain, xtest, ytest), 
                bulkrna = xbulk,
                name = 'Human Brain Cortex', verbose = True, 
                lr = 0.00001, batch_size = 256, epochs = epochs)

# train
sw.run()

Stablishing Early Stopping with patience 10
Stablishing Early Stopping with patience 10
Stablishing Early Stopping with patience 50


P1: Train MSE is: 0.000348, Test MSE is 0.001816:  12%|█▏        | 1758/15029 [00:26<03:17, 67.07it/s]


Early stopping condition achieved


P2: Train MSE is: 0.000274, Test MSE is: 0.000444:   1%|▏         | 212/15029 [00:33<38:40,  6.39it/s]


Early stopping condition achieved


  0%|          | 0/15029 [00:00<?, ?it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.